In [2]:
import pandas as pd
import json
import sklearn
import glob
import pickle
from sklearn.model_selection import train_test_split
from collections import Counter


pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
import sys
sys.path.insert(0, '../../style_generation_pipeline')

from data import *

In [4]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_2'

### Preparing data for explainability

#### Saving all data

In [10]:
all_documents = []
for file in glob.glob(path + '/HRS2_test_document_records/english/*/*.jsonl'):
    print(file)
    all_documents+= [json.loads(x) for x in open(file).readlines()]

/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.3_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.1_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.5_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.4_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.2_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/background/HRS2.5_background_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/background/HRS2.1_background_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/background/HRS2.2_back

In [4]:
with open(path+'/explainability/all_documents.jsonl', 'w') as output_file:
    for d in all_documents:
        json.dump(d, output_file)
        output_file.write('\n')

In [5]:
all_documents_df = pd.DataFrame(all_documents)
all_documents_df['authorID'] = all_documents_df.authorIDs.apply(lambda x:x[0])

In [22]:
all_documents_df.head()

,documentID,authorIDs,fullText,spanAttribution,isForeground,dateCollected,publiclyAvailable,collectionNum,source,deidentified,languages,lengthWords,dateCreated,timeCreated,sourceSpecific,authorID
0,3379c469-5fa9-5bc8-ab81-37c8ff266f45,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],Neutrogena Lathering Soap\n\nNeutrogena soap i...,[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],106,2024-02-06,22:17:39,"{'participantID': 'ER20001', 'internalDocID': 12}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
1,198e4e55-3ef9-50f7-b7f5-60f1f7952dd5,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],"Lululemon Align Tank Top, sonic pink, size 4\n...",[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],133,2024-02-06,22:17:40,"{'participantID': 'ER20001', 'internalDocID': 15}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
2,6c7bdb11-6021-56b1-b2cd-86d36639c9de,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],Retractable Double Dog Leash\n\nEverybody with...,[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],126,2024-02-06,22:17:40,"{'participantID': 'ER20001', 'internalDocID': 14}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
3,ab6c205a-7315-56ef-bcd5-116d9b0f3ec3,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],"Bic Erasable Pens, twelve pack, variety assort...",[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],125,2024-02-06,22:17:39,"{'participantID': 'ER20001', 'internalDocID': 13}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
4,1ffcebf2-70db-57cc-a366-f905427ed20c,[c41930b0-e8d8-57dd-9fc8-381c2275fcba],Tangle Teezer Detangling Hairbrush in Light Bl...,[{'authorID': 'c41930b0-e8d8-57dd-9fc8-381c227...,True,2024-03-01,False,HRS2.3,participant,True,[en],135,2024-03-01,15:50:15,"{'participantID': 'ER20022', 'internalDocID': ...",c41930b0-e8d8-57dd-9fc8-381c2275fcba


In [25]:
all_authors = all_documents_df["authorID"].unique()
training_atuhors, test_authors = sklearn.model_selection.train_test_split(all_authors, train_size=0.7, test_size=0.15, random_state=5, shuffle=True)

training_data = all_documents_df[all_documents_df['authorID'].isin(training_atuhors)]
test_data     = all_documents_df[all_documents_df['authorID'].isin(test_authors)]

In [26]:
training_data.to_json(path+'/explainability/train_authors.json')
test_data.to_json(path+'/explainability/test_authors.json')

#### Save documents that are appearing in cross genre data:

In [3]:
queries_and_cands = []
for file in glob.glob('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/*.jsonl'):
    print(file)
    queries_and_cands+= [json.loads(x) for x in open(file).readlines()]

/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/hrs_06-27-24_english_crossGenre-combined_TA2_input_candidates.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/hrs_06-27-24_english_crossGenre-combined_TA2_input_queries.jsonl


In [22]:
queries_and_cands_docs = [x['documentID'] for x in queries_and_cands]
all_data_docs =  [x['documentID'] for x in all_documents]

In [28]:
all_documents_in_cross_genre = [doc for doc in all_documents if doc['documentID'] in queries_and_cands_docs]

In [ ]:
with open(path+'/explainability/all_documents_in_cross_genre.jsonl', 'w') as output_file:
    for d in all_documents_in_cross_genre:
        json.dump(d, output_file)
        output_file.write('\n')

#### Saveing training and test authors for clustering

In [22]:
training_df,_,_ = get_aa_data_from_original_format('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/hrs_06-27-24_english_crossGenre-combined_TA2_input',
                                                  '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/groundtruth/hrs_06-27-24_english_crossGenre-combined_TA2')
training_df.reset_index(inplace=True)

In [23]:
training_df.head()

,index,authorID,fullText,documentID,source
0,0,4dc32c7f-2267-21cb-54fa-0172a1d0e8bb,Death Warrant\n\nThis could grow more on me ov...,0012f154-63c1-5425-9173-30f4bfbbcab4,candidates
1,1,907f636d-53c0-e846-b1a6-fa1a2e597a68,Got dismissed early from my first warehouse jo...,00186707-c27b-52d7-b42c-1c5c61be6c3c,candidates
2,2,d4717646-45ba-f675-a737-98015aeb72da,<PERSON>\n\nThe Podcast Macabre 2023 Horror Ch...,0022a50f-2da4-5c7a-91ad-be721c7b14c2,candidates
3,3,82675237-d89b-f1b4-0746-4cdee4c2b2a1,Developers vs. electromagnetism\n\nMore years ...,0027600a-97a5-59aa-b444-a2f98c02c34e,candidates
4,4,ce8ed462-f687-5d4a-93e8-cfa1a74cfcfb,"REMEMBERING OUR BELOVED\n\n<PERSON>, <DATE_TIM...",0029ea89-3146-5081-b43a-3144daf9eca2,candidates


In [44]:
training_authors = training_df.authorID.value_counts().to_dict()
training_authors = [a[0] for a in training_authors.items() if a[1] > 1]
training_authors, test_authors = train_test_split(training_authors, test_size=0.20)

In [45]:
new_training_df = training_df[training_df.authorID.isin(training_authors)]
test_df = training_df[training_df.authorID.isin(test_authors)]

In [46]:
print(len(new_training_df), len(test_df))

4653 1116


In [48]:
new_training_df.to_json(path+'/explainability/train_authors.json')
test_df.to_json(path+'/explainability/test_authors.json')

### Generated Style Features:

In [ ]:
!export TRANSFORMERS_CACHE="/mnt/swordfish-pool2/milad/hf-cache-new"
!export HF_DATASETS_CACHE="/mnt/swordfish-pool2/milad/hf-cache-new"
!export HF_HOME="/mnt/swordfish-pool2/milad/hf-cache-new"

!python generate_styles.py --data-dir /mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/all_documents_in_cross_genre.jsonl \
                          --generator-model llama3 \
                          --shortener-model openai:gpt-3.5-turbo \
                          --style-threshold 2 \
                          --max-new-tokens 512 \
                          --device 0 1 \

In [47]:
df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv')
feat_to_ling_lvl = json.load(open('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/feats_to_ling_lvl.json'))

In [48]:
df['ling_lvl'] = df.original_attribute_name.apply(lambda x: feat_to_ling_lvl[x] if x in feat_to_ling_lvl else 'other')

In [49]:
print(df.documentID.nunique())
print(df['shortend_attribute_name.v2'].nunique(), df.aggregated_name.nunique())
print(df.ling_lvl.nunique())

6886
3228 3120
4


In [50]:
df.head()

,final_attribute_name,original_attribute_name,documentID,shortend_attribute_name.v1,shortend_attribute_name.v2,aggregated_name,ling_lvl
0,simple sentence structures,"The author uses simple sentence structures, wi...",3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author utilizes straightforward sentence s...,The author uses simple sentence structures.,The author uses simple sentence structures.,Morphological Level
1,diverse sentence lengths,The author employs a mix of short and mediumle...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author uses a variety of sentence lengths ...,The writer employs diverse sentence lengths in...,The writer employs diverse sentence lengths in...,Morphological Level
2,past tense,The author favors the use of present tense ver...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,Rewritten: The author prefers present tense ve...,Rewritten: The author favors present tense ove...,Rewritten: The author favors present tense ove...,Morphological Level
3,consistent pattern,The author uses a consistent verb conjugation ...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author maintains a consistent verb conjuga...,Rewritten: The author follows a consistent pat...,Rewritten: The author follows a consistent pat...,Morphological Level
4,verb conjugation,The author uses a consistent verb conjugation ...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author maintains a consistent verb conjuga...,Rewritten: The author follows a consistent pat...,Rewritten: The author follows a consistent pat...,Morphological Level


In [52]:
df.groupby('final_attribute_name').agg({'documentID': lambda x: len(x), 'ling_lvl': lambda x: list(x)[0]}).reset_index().to_csv(path + '/explainability/final_attribute_name.csv')
df.groupby('aggregated_name').agg({'documentID': lambda x: len(x), 'ling_lvl': lambda x: list(x)[0]}).reset_index().to_csv(path + '/explainability/aggregated_name.csv')

In [96]:
g_df = df.groupby('final_attribute_name').agg({'original_attribute_name': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'shortend_attribute_name.v1': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'shortend_attribute_name.v2': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'aggregated_name': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'documentID': lambda x: len(x),
                                               'ling_lvl': lambda x: list(x)
                                    }).reset_index()

g_df['final_attribute_ling_lvl'] = g_df['ling_lvl'].apply(lambda x: Counter(x).most_common(1)[0][0])

In [99]:
g_df.sort_values('final_attribute_name').head(n=10)

,final_attribute_name,original_attribute_name,shortend_attribute_name.v1,shortend_attribute_name.v2,aggregated_name,documentID,ling_lvl,final_attribute_ling_lvl
0,abstract concepts,{'The author employs a range of abstract conce...,{'The author uses abstract concepts to analyze...,{'Rewritten: The writer explores deep ideas us...,{'Rewritten: The writer explores deep ideas us...,78,"[Semantic Level, Semantic Level, Semantic Leve...",Semantic Level
1,abstract concepts rather,{'The author tends to focus on the intellectua...,{'The author emphasizes abstract concepts over...,{'The emphasis is on abstract concepts rather ...,{'The emphasis is on abstract concepts rather ...,5,"[Semantic Level, Semantic Level, Semantic Leve...",Semantic Level
2,abstract ideas,"{'The author uses a high level of abstraction,...",{'The author utilizes abstract concepts to del...,{'The author uses a moderate level of abstract...,{'The writer uses abstract ideas to explore th...,47,"[Semantic Level, Semantic Level, Semantic Leve...",Semantic Level
3,abstract nouns,{'The author uses a mix of concrete and abstra...,{'The author uses a variety of concrete and ab...,{'Rewritten: The author incorporates a mix of ...,{'Rewritten: The author incorporates a mix of ...,4,"[Morphological Level, Morphological Level, Syn...",Morphological Level
4,academic style,"{'The author uses a formal, academic tone, wit...",{'The author employs a scholarly tone to commu...,{'The text uses an academic style to convey co...,{'The text uses an academic style to convey co...,3,"[Syntactic Level, Syntactic Level, Syntactic L...",Syntactic Level
5,accessible language,{'The author uses a moderate level of vocabula...,{'The author aims to establish a bond with the...,{'The author seeks to engage the reader throug...,{'The author seeks to engage the reader throug...,60,"[Morphological Level, Semantic Level, Morpholo...",Discourse Level
6,active manner,"{'The author uses active voice, with a focus o...",{'The writing emphasizes the narrator's action...,{'Rewritten: The writing focuses on the narrat...,{'Rewritten: The writing focuses on the narrat...,3,"[Syntactic Level, Syntactic Level, Syntactic L...",Syntactic Level
7,active voice,{'The author uses active voice to create a sen...,{'The author often employs active voice to emp...,{'The writing style often features active voic...,{'The writing style often features active voic...,285,"[Morphological Level, Morphological Level, Mor...",Syntactic Level
8,active voice enhances,{'The author uses active voice to convey a sen...,{'The use of active voice conveys a sense of e...,{'Rewritten: Active voice enhances engagement....,{'Rewritten: Active voice enhances engagement....,5,"[Syntactic Level, Syntactic Level, Syntactic L...",Syntactic Level
9,additional clauses,{'The author uses relative clauses to provide ...,{'The author utilizes subordinate clauses for ...,{'The writer uses additional clauses for clari...,{'The writer uses additional clauses for clari...,550,"[Syntactic Level, Syntactic Level, Syntactic L...",Syntactic Level


In [71]:
g_df.to_json(path + '/explainability/style_features_corpus.json', orient='records', indent=2)

In [73]:
print(df.aggregated_name.nunique())
print(df.final_attribute_name.nunique())
g_df.final_attribute_ling_lvl.value_counts()

3120
1014


Semantic Level         308
Discourse Level        272
Syntactic Level        265
Morphological Level    169
Name: final_attribute_ling_lvl, dtype: int64

#### Merge the Gram2vec style feats to our LLM-based style feats:

In [28]:
llm_based_df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv')
llm_based_df = llm_based_df.groupby('documentID').agg({'final_attribute_name': lambda x: list(x)}).reset_index()

In [30]:
gram2vec_df   = pd.read_json('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/gram2vec_top_features.jsonl', lines=True)
gram2vec_dict = {x[0]: x[1] for x in zip(gram2vec_df.documentID.tolist(), gram2vec_df.gram2vec_feats.tolist())} 
gram2vec_feats = set([x for feats in gram2vec.values() for x in feats])

In [37]:
gram2vec_df.head()

,documentID,gram2vec_feats
0,3379c469-5fa9-5bc8-ab81-37c8ff266f45,"[Part-of-Speech Unigram:Noun, Morphology Tag:S..."
1,198e4e55-3ef9-50f7-b7f5-60f1f7952dd5,"[Morphology Tag:Singular number, Part-of-Speec..."
2,ab6c205a-7315-56ef-bcd5-116d9b0f3ec3,"[Part-of-Speech Unigram:Noun, Morphology Tag:S..."
3,21c85995-32d3-53f3-a316-4f934afae2b1,"[Part-of-Speech Unigram:Noun, Morphology Tag:S..."
4,572eb600-05b3-52d5-8332-c8a30f336690,"[Part-of-Speech Unigram:Noun, Morphology Tag:S..."


In [31]:
llm_based_df['final_attribute_name'] = llm_based_df.apply(lambda x: x['final_attribute_name'] + gram2vec_dict[x['documentID']], axis=1)

In [39]:
llm_based_df = llm_based_df.explode(['final_attribute_name'])
gram2vec_df  = gram2vec_df.explode(['gram2vec_feats'])

In [40]:
llm_based_df.to_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/llm_and_gram2vec_feats.csv', index=False)
gram2vec_df.to_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/gram2vec_feats.csv', index=False)

### Consturcted Interpretable space:

In [152]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/test_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/" \
--model aa_model-luar \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv"

Testing Different Epsilon Values:  34%|###      | 34/99 [03:47<07:18,  6.74s/it]^C
Traceback (most recent call last):                                              
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 250, in <module>
    main(args)
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 107, in main
    best_eps, train_df_by_author = find_best_eps()
                                   ^^^^^^^^^^^^^^^
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 94, in find_best_eps
    eps_to_performance[eps] = compute_model_performance(model, test_df, new_bases)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ma4608/style_generation_pipeline/metrics/__init__.py", line 75, in compute_model_performance
    y_interp_score = [
                     ^
  File "/home/ma4608/style_generation_pipeline/metrics/__init__.py", line 78, in <lis

In [43]:
# clustering considering both gram2vec and llm-based style feats
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/test_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/llm_and_gram2vec/" \
--model aa_model-luar \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/llm_and_gram2vec_feats.csv" \
--style_feat_column 'final_attribute_name' \
--top_k_feats 10\
--eps 0.08

In [44]:
# clustering considering gram2vec style feats
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/test_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/gram2vec/" \
--model aa_model-luar \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/gram2vec_feats.csv" \
--style_feat_column 'gram2vec_feats' \
--top_k_feats 10\
--eps 0.08

In [46]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/test_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/llm" \
--model aa_model-luar \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv" \
--style_feat_column 'final_attribute_name' \
--top_k_feats 10\
--eps 0.08

In [153]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/test_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/" \
--model aa_model-luar \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv" \
--style_feat_column 'aggregated_name' \
--summarize_cluster_reps\
--top_k_feats 10\
--eps 0.08

Summarizing styles of interpretable dimensions
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ma4608/.cache/huggingface/token
Login successful
[('The author uses complex sentence structures.', 1455.4088296205819), ('The text features informal language.', 1344.8628631819436), ('The writer employs diverse sentence lengths in their work.', 1320.6544008550543), ('The author employs diverse sentence structures.', 1274.9742628495105), ('The writing style is informal.', 1273.9444722086032), ('The writer employs diverse sentence structures in their work.', 1185.3959836034117), ('The text includes casual language.', 1156.6591816164225), ('Rewritten: Varying sentence lengths enhances the overall rhythm and flow.', 1119.1125372506235), ('The author ut

In [156]:
df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.pkl')

In [160]:
vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/interpretable_space.pkl')

In [161]:
len(vector_to_style_dist)

15

In [162]:
[(key,sorted(x[1].items(), key=lambda i:-i[1])[:10]) for key, x in vector_to_style_dist.items()]

[(-1,
  [('The author uses complex sentence structures.', 1455.4088296205819),
   ('The text features informal language.', 1344.8628631819436),
   ('The writer employs diverse sentence lengths in their work.',
    1320.6544008550543),
   ('The author employs diverse sentence structures.', 1274.9742628495105),
   ('The writing style is informal.', 1273.9444722086032),
   ('The writer employs diverse sentence structures in their work.',
    1185.3959836034117),
   ('The text includes casual language.', 1156.6591816164225),
   ('Rewritten: Varying sentence lengths enhances the overall rhythm and flow.',
    1119.1125372506235),
   ('The author utilizes different sentence structures in their writing.',
    1117.8820055629183),
   ('The author usually follows a particular structure in their writing.',
    1071.0835997950185)]),
 (0,
  [('The author uses precise language in a formal manner.', 89.38880339383041),
   ('The author uses additional clauses to provide more information.',
    81.11

In [163]:
[(key,x[2]) for key, x in vector_to_style_dist.items()]

[(-1,
  'The author is skilled in using diverse sentence structures and lengths to enhance the rhythm and flow of their writing. They also incorporate informal language to create a unique writing style.'),
 (0,
  'The author is skilled in using precise language and diverse sentence structures to convey a formal and professional tone. They maintain consistency in verb tense usage throughout their writing, ensuring effective communication and clarity.'),
 (1,
  'The author frequently incorporates additional information through relative clauses, employs diverse sentence structures for improved flow, and uses different verb tenses effectively for a cohesive narrative.'),
 (2,
  'The author is skilled in employing diverse sentence lengths, informal language, and literary devices to create a dynamic and engaging writing style.'),
 (3,
  'The author avoids complex sentence structures and instead opts for straightforward ones. They incorporate various types of clauses for additional informatio